### ConditionGAN實作
給定condition,讓機器生成和條件相關的圖片
[reference](https://chenllliang.github.io/2020/02/03/cGAN/)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip /content/drive/MyDrive/CGAN_data/data.zip

串流輸出內容已截斷至最後 5000 行。
  inflating: data/faces/29965.jpg    
  inflating: __MACOSX/data/faces/._29965.jpg  
  inflating: data/faces/5472.jpg     
  inflating: __MACOSX/data/faces/._5472.jpg  
  inflating: data/faces/18206.jpg    
  inflating: __MACOSX/data/faces/._18206.jpg  
  inflating: data/faces/3003.jpg     
  inflating: __MACOSX/data/faces/._3003.jpg  
  inflating: data/faces/10882.jpg    
  inflating: __MACOSX/data/faces/._10882.jpg  
  inflating: data/faces/15722.jpg    
  inflating: __MACOSX/data/faces/._15722.jpg  
  inflating: data/faces/8156.jpg     
  inflating: __MACOSX/data/faces/._8156.jpg  
  inflating: data/faces/7265.jpg     
  inflating: __MACOSX/data/faces/._7265.jpg  
  inflating: data/faces/380.jpg      
  inflating: __MACOSX/data/faces/._380.jpg  
  inflating: data/faces/1614.jpg     
  inflating: __MACOSX/data/faces/._1614.jpg  
  inflating: data/faces/25839.jpg    
  inflating: __MACOSX/data/faces/._25839.jpg  
  inflating: data/faces/8142.jpg     
  inflating: 

In [ ]:
!unzip /content/drive/MyDrive/CGAN_data/data.zip -d /content/drive/MyDrive/CGAN_data

串流輸出內容已截斷至最後 5000 行。
  inflating: /content/drive/MyDrive/CGAN_data/data/faces/29965.jpg  
  inflating: /content/drive/MyDrive/CGAN_data/__MACOSX/data/faces/._29965.jpg  
  inflating: /content/drive/MyDrive/CGAN_data/data/faces/5472.jpg  
  inflating: /content/drive/MyDrive/CGAN_data/__MACOSX/data/faces/._5472.jpg  
  inflating: /content/drive/MyDrive/CGAN_data/data/faces/18206.jpg  
  inflating: /content/drive/MyDrive/CGAN_data/__MACOSX/data/faces/._18206.jpg  
  inflating: /content/drive/MyDrive/CGAN_data/data/faces/3003.jpg  
  inflating: /content/drive/MyDrive/CGAN_data/__MACOSX/data/faces/._3003.jpg  
  inflating: /content/drive/MyDrive/CGAN_data/data/faces/10882.jpg  
  inflating: /content/drive/MyDrive/CGAN_data/__MACOSX/data/faces/._10882.jpg  
  inflating: /content/drive/MyDrive/CGAN_data/data/faces/15722.jpg  
  inflating: /content/drive/MyDrive/CGAN_data/__MACOSX/data/faces/._15722.jpg  
  inflating: /content/drive/MyDrive/CGAN_data/data/faces/8156.jpg  
  inflating: /content

In [ ]:
import zipfile
dir = '/content/drive/My Drive/CGAN_data/data.zip'
zp = zipfile.ZipFile(dir)
for f in zp.namelist():
  zp.extract(f,'/content')
zp.close()

In [ ]:
import numpy as np
import re

f=open("/content/drive/MyDrive/CGAN_data/train_tag_dict.txt","r")
data =eval(f.read())


class Voc:
    def __init__(self, name):
        self.name = name
        self.trimmed = False
        self.word2index = {}
        self.word2count = {}
        self.index2word = {}
        self.num_words = 0

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            if word != "hair" and word != "eyes" and word != "and":
                self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.num_words
            self.word2count[word] = 1
            self.index2word[self.num_words] = word
            self.num_words += 1
        else:
            self.word2count[word] += 1

    # 删除低于特定计数阈值的词，降低模型复杂度
    def trim(self, min_count):
        if self.trimmed:
            return
        self.trimmed = True
        keep_words = []
        abondon_words = []
        for k, v in self.word2count.items():
            if v >= min_count:
                keep_words.append(k)
            else:
                abondon_words.append(k)

        print('keep_words {} / {} = {:.4f}'.format(len(keep_words), len(self.word2index),
                                                   len(keep_words) / len(self.word2index)))

        self.word2index = {}
        self.word2count = {}
        self.index2word = {}
        self.num_words = 0

        for word in keep_words:
            self.addWord(word)

        return keep_words,abondon_words

    def split_description(self, sentence):
        q = sentence.split(" ")
        hair = []
        eyes = []
        if "and" in q:
            del q[q.index("and")]
            hair_index = q.index("hair")
            eyes_index = q.index("eyes")
            if hair_index < eyes_index:
                hair = q[0:hair_index]
                eyes = q[hair_index + 1:eyes_index]
            else:
                eyes = q[0:eyes_index]
                hair = q[eyes_index + 1:hair_index]

            return hair, eyes

        if "hair" in q:
            del q[q.index("hair")]
            hair = q
        if "eyes" in q:
            del q[q.index("eyes")]
            eyes = q

        return hair, eyes

    def generate_2_vectors(self, hair, eyes): #one hot encoding for conditions
        h_vector = np.zeros(self.num_words)
        e_vector = np.zeros(self.num_words)

        for i in hair:
            h_vector[self.word2index[i]] = 1 #不同的hair不同bits會有不同的encoding
        for i in eyes:
            e_vector[self.word2index[i]] = 1

        return h_vector, e_vector

    def generate_final_vector(self,description):
        hair,eyes = self.split_description(description)
        h_vector,e_vector = self.generate_2_vectors(hair,eyes)
        return np.concatenate((h_vector,e_vector),axis=0)#所有頭髮和眼睛文字條件的one hot encoding concate在一起



voc = Voc("d_vector")
for key in data:
    voc.addSentence(data[key])

keep_words,abondon_words=voc.trim(100)


def set_cross(the_set,sentence):
    for i in the_set:
        if i in sentence:
            return True
    return False

b_data=data.copy()

for key in b_data:
    if set_cross(abondon_words,b_data[key]):
        del(data[key])






import os
#删除未知标签的文件

def delete_imgs(data):
    required_img = list(data.keys())

    for i in range(33430):
        i=i+1
        if i not in required_img:
            path = r"/content/drive/MyDrive/CGAN_data/__MACOSX/data/faces/"+str(i)+".jpg"
            os.remove(path)

f=open("final_train_tag_dict.txt","w")
f.write(str(data))
f.close()

keep_words 16 / 18 = 0.8889


### Load Data
將動漫圖片載入
** error:condition index(txt裡的)對不上圖片的檔名= =???

In [ ]:
from torch.utils import data
import numpy as np
from PIL import Image

In [ ]:
class face_dataset():
    def __init__(self):
      #self.f2 = '/content/__MACOSX/data/faces'
      self.file_path = '/content/drive/MyDrive/CGAN_data/__MACOSX/data/faces/'
      
      f=open("/content/drive/MyDrive/CGAN_data/final_train_tag_dict.txt","r")
      self.label_dict=eval(f.read())
      
      #self.file_path2 = '/content/__MACOSX/data/faces'
      f.close()

    def __getitem__(self,index):
      label = list(self.label_dict.values())[index-1]
      img_id = list(self.label_dict.keys())[index-1]
      print(img_id)
      img_path = self.file_path+str(img_id)+".jpg"
      img = np.array(Image.open(img_path))
      return img,label

    def __len__(self):
      return len(self.label_dict)

a = face_dataset()

img,label = a[1]

print(img)

2


FileNotFoundError: ignored

### Model for CGAN
[BN詳細介紹](https://blog.csdn.net/qq_39777550/article/details/108038677)

In [ ]:
import torch.nn as nn
import torch

class CNN_Generator(nn.Module):
    def __init__(self,noize_dim,label_dim,num_feature):
        super(CNN_Generator, self).__init__()
        self.fc = nn.Linear(noize_dim+label_dim,num_feature)  # batch, 15*192*192
        #第一层全连接，首先将噪声和标签信息拼接，映射成一个高维的向量
        self.br = nn.Sequential(
            nn.BatchNorm2d(15),#input channel size
            nn.ReLU(True)
        )
        self.downsample1 = nn.Sequential(
            nn.Conv2d(15, 50, 3, stride=1, padding=1),  # batch, 50, 192, 192
            nn.BatchNorm2d(50),
            nn.ReLU(True)
        )
        self.downsample2 = nn.Sequential(
            nn.Conv2d(50, 25, 3, stride=1, padding=1),  # batch, 25, 192, 192
            nn.BatchNorm2d(25),
            nn.ReLU(True)
        )
        self.downsample3 = nn.Sequential(
            nn.Conv2d(25, 3, 2, stride=2),  # batch, 3, 96, 96
            nn.Tanh()
        )

    def forward(self, x):
        x = self.fc(x)
        x = x.view(x.size(0), 15, 192, 192)
        x = self.br(x)
        x = self.downsample1(x)
        x = self.downsample2(x)
        x = self.downsample3(x)
        return x


class CNN_Discriminator(nn.Module):
    def __init__(self,label_dim):
        super(CNN_Discriminator, self).__init__()

        self.label_embedding = nn.Linear(label_dim,96*96)

        self.conv1 = nn.Sequential(
            nn.Conv2d(in_channels=4, out_channels=32, kernel_size=5, padding=2),  # batch, 32, 96，96,
            nn.LeakyReLU(0.2, True),
            nn.AvgPool2d(2, stride=2),  # batch, 32, 48, 48
        )
        self.conv2 = nn.Sequential(
            nn.Conv2d(32, 64, 5, padding=2),  # batch, 64, 48, 48
            nn.LeakyReLU(0.2, True),
            nn.AvgPool2d(2, stride=3)  # batch, 64, 16, 16
        )
        self.fc = nn.Sequential(
            nn.Linear(64 * 16 * 16, 1024),
            nn.LeakyReLU(0.2, True),
            nn.Linear(1024, 1),
            nn.Sigmoid()
        )

    def forward(self, x, label):
        '''
        x: batch, width, height, channel=3
        '''
        label = self.label_embedding(label).view(-1,1,96,96)
        x = torch.cat([x,label],1)
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


### main實際運行處：
***一些註解***
1. 關於Variables:
The original purpose of Variables was to be able to use automatic differentiation (Tensor form)


In [ ]:

import torch
import numpy as np
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable
import os
#import cGAN_Model
#from Onehot_embedding import Voc
#import yaml
import random

def to_img(x):
	out = 0.5 * (x + 1)
	out = out.clamp(0, 1)
	out = x.view(-1, 3, 96, 96)
	return out


# file=open('config.yml')
# config = yaml.load(file,Loader=yaml.Loader)
# file.close()




#导入经过处理的label原始信息
f=open("final_train_tag_dict.txt","r")
data =eval(f.read())


# print("Showing the first 5 descriptions")
# #展示前5条
# for i in range(5):
#     print("img_index",list(data.keys())[i],"description:",data[list(data.keys())[i]])


#生成label向量
voc = Voc("d_vector")
for key in data:
	voc.addSentence(data[key])
voc.trim(100)

# #展示5条onehot编码的向量
# print("Showing the one_hot vector for the first 5 descriptions")
# for i in range(5):
#     print("img_index",list(data.keys())[i],"description:",data[list(data.keys())[i]])
#     print("onehot_vector:",voc.generate_final_vector(data[list(data.keys())[i]]))


batch_size= 500
num_epoch= 5
z_dimension= 512

#統一把圖片轉成tensor,並做標準化
img_transform = transforms.Compose([
	transforms.ToTensor(),
	transforms.Normalize([0.5], [0.5])
])


FaceDataset = datasets.ImageFolder('/content/drive/MyDrive/CGAN_data/__MACOSX/data/', transform=img_transform) # 数据路径
dataloader = torch.utils.data.DataLoader(FaceDataset,
									 batch_size=batch_size, # 批量大小
									 shuffle=False, # 不要乱序
									 num_workers=8 # 多进程
									 )
label = list(data.values())
num_samples = len(label)

label_vectors=[]
for i in label:
	label_vectors.append(torch.from_numpy(voc.generate_final_vector(i)))



batch_vectors=torch.cat(label_vectors[0:10],0)
batch_vectors=batch_vectors.view(-1,32).float()


G = CNN_Generator(z_dimension,32,15*192*192)
D = CNN_Discriminator(32)

if torch.cuda.is_available():
	D = D.cuda()
	G = G.cuda()

device = torch.device("cuda:2" if torch.cuda.is_available() else "cpu")
# Binary cross entropy loss and optimizer
criterion = nn.BCELoss()
d_optimizer = torch.optim.Adam(D.parameters(), lr=0.0002)
g_optimizer = torch.optim.Adam(G.parameters(), lr=0.0002)





for epoch in range(num_epoch):
	for i, (img,_) in enumerate(dataloader):

		if (i+1)*batch_size < num_samples:
			batch_vectors=torch.cat((label_vectors[i*batch_size:(i+1)*batch_size]),0)
			batch_vectors=batch_vectors.view(-1,32).float()
		else:
			batch_vectors=torch.cat((label_vectors[i*batch_size:-1]),0)
			batch_vectors=batch_vectors.view(-1,32).float()

		if torch.cuda.is_available():
			torch.cuda.empty_cache()

		num_img = img.size(0)
		#train discriminator
		# compute loss of real_matched_img
    #(真實圖片,正確文字敘述)
		img = img.view(num_img,3,96,96)
		real_img = Variable(img).to(device) #真實圖片load進來
		real_label = Variable(torch.ones(num_img)).to(device)#real label會assign成1,算loss的時候要讓正確的lable&image對應的值和'1'越接近越好
		fake_label = Variable(torch.zeros(num_img)).to(device)
		batch_vectors = Variable(batch_vectors).to(device)
		matched_real_out = D(real_img,batch_vectors)
		d_loss_matched_real = criterion(matched_real_out, real_label)#make matched pair as close as 1
		matched_real_scores = matched_real_out  # closer to 1 means better

		# compute loss of fake_matched_img
    #(假的圖片,正確文字敘述)
		z = Variable(torch.randn(num_img, z_dimension)).to(device)#隨機初始化noisr,丟給generator
		z = torch.cat((z,batch_vectors),axis=1).to(device) #encode好的condition(labels)一起丟給D
		fake_img = G(z)
		matched_fake_out = D(fake_img,batch_vectors)
		d_loss_matched_fake = criterion(matched_fake_out, fake_label)#假圖片真condition的pair輸出要越接近0越好(should have low score!)
		matched_fake_out_scores = matched_fake_out  # closer to 0 means better

		# compute loss of real_unmatched_img
    #(真的圖片,錯誤文字敘述)
		rand_label_vectors=random.sample(label_vectors,num_img)#隨機sample一些encode過後的文字condition出來
		rand_batch_vectors=torch.cat((rand_label_vectors[:]),0)
		rand_batch_vectors=rand_batch_vectors.view(-1,32).float().to(device)


		z = Variable(torch.randn(num_img, z_dimension)).to(device)
		z = torch.cat((z,rand_batch_vectors),axis=1).to(device)
		fake_img = G(z)#隨機sample出來亂掉的condition+noise一起給Ｇ
		unmatched_real_out = D(fake_img,batch_vectors)
		d_loss_unmatched_real = criterion(unmatched_real_out, fake_label)
		unmatched_real_out_scores = unmatched_real_out  # closer to 0 means better

		# bp and optimize
		d_loss = d_loss_matched_real + d_loss_matched_fake + d_loss_unmatched_real
		d_optimizer.zero_grad()
		d_loss.backward()
		d_optimizer.step()

		# ===============train generator
		# compute loss of fake_img
		# compute loss of fake_matched_img
    #G希望根據真實condition batch vector,產出以noise z為輸入的假圖片，讓他越貼近condition越好
		z = Variable(torch.randn(num_img, z_dimension)).to(device)
		z = torch.cat((z,batch_vectors),axis=1).to(device)
		fake_img = G(z)
		matched_fake_out = D(fake_img,batch_vectors)
		matched_fake_out_scores = matched_fake_out

		g_loss = criterion(matched_fake_out,real_label)#希望Ｄ給的分數越接近一月好（表示和真實圖片＋condition都有match!）

		# bp and optimize
		g_optimizer.zero_grad()
		g_loss.backward()
		g_optimizer.step()


		print('Epoch [{}/{}], Batch {},d_loss: {:.6f}, g_loss: {:.6f} '
				  .format(
				epoch, num_epoch,i,d_loss.data, g_loss.data,
				))
		if epoch == 0:
			real_images = to_img(real_img.cpu().data)
			save_image(real_images, './img/real_images.png')

		fake_images = to_img(fake_img.cpu().data)
		save_image(fake_images, './img/fake_images-{}.png'.format(i+(epoch-1)*batch_size))

torch.save(G.state_dict(), './generator.pth')
torch.save(D.state_dict(), './discriminator.pth')
